In [ ]:
import pandas as pd
from scipy.signal import butter, filtfilt

In [ ]:
data_file = "Minitab.csv"
index = "time_index.csv"
skip = 0
df = pd.read_csv(data_file)
times = pd.read_csv(index)

In [ ]:
fs = 100;   fc = 1;     order = 4
nyq = fs * 0.5;         low = fc / nyq
b, a = butter(order, low, btype='low', analog=False)

In [ ]:
data_col = df.columns[10:19]
for col in data_col:
    col_new = col + " filtered"
    df[col_new] = pd.to_numeric(df[col], errors='coerce')
    df[col_new] = df[col_new].interpolate()
    df[col_new] = df.groupby('Run')[col_new].transform(lambda x: filtfilt(b, a, x))

In [ ]:
if skip == 0:
    runs = df.groupby('Run')
    rows = times.iterrows()
    results = pd.DataFrame(columns=df.columns)
    results['cavitation mode'] = []
    for row in rows:
        i = 0
        for col in row[1]:
            cav = pd.Series([times.columns[i]], index=['cavitation mode'])
            series = runs.get_group(row[0]+1).iloc[col*100].append(cav)
            results = results.append(series, ignore_index=True)
            i += 1

    results.to_csv("ABB/ABB Data Points.csv", index=False)

In [ ]:
df.to_csv("ABB/ABB Data.csv", index=False)